In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san antonio de pale,-1.4014,5.6325,25.71,79,100,6.13,GQ,1685417879
1,1,papatowai,-46.5619,169.4708,8.24,90,100,6.08,NZ,1685417880
2,2,kristiansund,63.1115,7.7320,9.03,93,100,7.72,NO,1685417880
3,3,yarovoye,52.9273,78.5800,18.08,42,0,5.29,RU,1685417641
4,4,zakamensk,50.3741,103.2863,17.75,37,100,4.30,RU,1685417880


In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 26) & (city_data_df["Max Temp"] > 20) \
    & (city_data_df["Wind Speed"] < 3.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,miandrivazo,-19.5167,45.4667,21.84,59,0,1.03,MG,1685417887
162,162,sweet home,44.3976,-122.7362,22.62,89,0,1.00,US,1685417921
214,214,morondava,-20.2833,44.2833,22.28,54,0,2.94,MG,1685417933
287,287,kemin,42.7846,75.6892,21.29,36,0,0.92,KG,1685417949
298,298,san sebastian de la gomera,28.0916,-17.1133,20.21,72,0,1.54,ES,1685417952
362,362,al kharijah,25.4514,30.5464,25.38,32,0,0.89,EG,1685417971
440,440,birjand,32.8663,59.2211,22.26,15,0,3.09,IR,1685417992
470,470,kosonsoy,41.2561,71.5508,23.31,21,0,1.29,UZ,1685417998
507,507,at taj,24.2049,23.2857,22.05,26,0,3.42,LY,1685418012
512,512,blytheville,35.9273,-89.9190,21.99,53,0,1.54,US,1685418013


In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,miandrivazo,MG,-19.5167,45.4667,59,
162,sweet home,US,44.3976,-122.7362,89,
214,morondava,MG,-20.2833,44.2833,54,
287,kemin,KG,42.7846,75.6892,36,
298,san sebastian de la gomera,ES,28.0916,-17.1133,72,
362,al kharijah,EG,25.4514,30.5464,32,
440,birjand,IR,32.8663,59.2211,15,
470,kosonsoy,UZ,41.2561,71.5508,21,
507,at taj,LY,24.2049,23.2857,26,
512,blytheville,US,35.9273,-89.9190,53,


In [7]:
 # Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "no hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
miandrivazo - nearest hotel: Baobab
sweet home - nearest hotel: No hotel found
morondava - nearest hotel: Hotel Sharon Menabe
kemin - nearest hotel: Кемин
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
al kharijah - nearest hotel: Kharga Hotel
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
kosonsoy - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
blytheville - nearest hotel: Fairview Suites
aswan - nearest hotel: Yaseen hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
32,miandrivazo,MG,-19.5167,45.4667,59,Baobab
162,sweet home,US,44.3976,-122.7362,89,No hotel found
214,morondava,MG,-20.2833,44.2833,54,Hotel Sharon Menabe
287,kemin,KG,42.7846,75.6892,36,Кемин
298,san sebastian de la gomera,ES,28.0916,-17.1133,72,Hotel Villa Gomera
362,al kharijah,EG,25.4514,30.5464,32,Kharga Hotel
440,birjand,IR,32.8663,59.2211,15,مهمانسرای جهانگردی بیرجند
470,kosonsoy,UZ,41.2561,71.5508,21,No hotel found
507,at taj,LY,24.2049,23.2857,26,No hotel found
512,blytheville,US,35.9273,-89.9190,53,Fairview Suites


In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)